## First Layer

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

import datetime
import time
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
from datetime import date
date_today = date.today()

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
driver = webdriver.Chrome(options=options, executable_path=r'/Users/ricardolu/Desktop/Design lab project/chromedriver')
driver.set_window_position(0, 0)
driver.set_window_size(1248, 768)

driver.get("https://us.topman.com/en/tmus/category/clothing-172005?currentPage=1")
time.sleep(1.5)



counter = 1

for i in range(1,3):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:

        # Scroll down to bottom
        driver.execute_script("window.scrollBy(0, 768);")

        # Wait to load page
        time.sleep(.2)

        # Calculate new scroll height and compare with last scroll height
        new_height = 768 * counter
        if new_height >= last_height:
            time.sleep(5.5)
            break

        counter += 1

soup = BeautifulSoup(driver.page_source,'html.parser')

In [2]:
product_name = []
original_price = []
sale_price = []
product_url = []
image_url = []
brand = []
product_type = []
timestamp = []


for i in soup.find_all('div',{'class':'Product Product--col3'}):
    if i.find('a',{'class':'Product-nameLink'}) != None:
        product_name.append(i.find('a',{'class':'Product-nameLink'}).text.strip())
    else:
        product_name.append(None)

    if i.find('span',{'class':'Price notranslate'}) != None:
        sale_price.append((i.find('span',{'class':'Price notranslate'})).text.strip())
    else:
        sale_price.append(None)
        
    original_price.append(None)

    if i.find('a',{'class':'Product-link'}) != None:
        product_url.append("https://us.topman.com"+i.find('a',{'class':'Product-link'}).get('href'))
    else:
        product_url.append(None)

    if i.find('img',{'class':'ProductImages-image ProductImages-mobile--showing'}) !=None:
        image_url.append("https:"+(i.find('img',{'class':'ProductImages-image ProductImages-mobile--showing'})).get('src'))
    else:
        image_url.append(None)
    brand.append("topshop")
    product_type.append("Men's Clothing")
    timestamp.append(date_today)

In [3]:
column_names = ["product_name", "original_price", "sale_price",'product_link', 'image_link']
df = pd.DataFrame(columns = column_names)

df['product_name'] = product_name
df['original_price'] = original_price
df['sale_price'] = sale_price
df['image_link'] = image_url
df['product_link'] = product_url

In [4]:
df

,product_name,original_price,sale_price,product_link,image_link
0,Camel Classic T-Shirt,None,$10.00,https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM71B36ACAM...
1,Black Spray On Skinny Jeans,None,$60.00,https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM69F55NBLK...
2,CONSIDERED Burgundy Twist Essential Knitted Sw...,None,$35.00,https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM81B14VBRG...
3,Black Classic T-Shirt,None,$10.00,https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM71B01TBLK...
4,Black Stretch Skinny Jeans,None,$60.00,https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM69F54QBLK...
...,...,...,...,...,...
595,Gray Skater Jeans,None,$65.00,https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM69H98TGRY...
596,Yellow Waffle Docker Beanie,None,$20.00,https://us.topman.com/en/tmus/product/shoes-an...,https://images.topman.com/i/TopMan/TM56D11CYLW...
597,Black and White Blur Floral Print Slim Shirt,None,$55.00,https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM83U10UBLK...
598,Black MA2 Bomber Jacket,None,$95.00,https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM64J15UBLK...


## Second Layer

In [6]:
product_description = []
material = []
picture_name = []
color = []
counter = 1
driver = webdriver.Chrome(options=options, executable_path=r'/Users/ricardolu/Desktop/Design lab project/chromedriver')

for link in product_url:
    driver.get(link)
    time.sleep(1)

    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    if soup.find('h1',{'class':'ProductDetail-title'}) != None:
        if soup.find('div',{'class':'ProductDescription-content'}).find('strong') != None:
            product_description.append(soup.find('div',{'class':'ProductDescription-content'}).find('strong').text.strip())
        elif soup.find('div',{'class':'ProductDescription-content'}) != None:
            description_item = []
            for j in soup.find('div',{'class':'ProductDescription-content'}).find_all('li'):
                description_item.append(j.text.strip())
            product_description.append(description_item)
        else:
            product_description.append(None)

        material_item = []
        if soup.find('div',{'class':'ProductDescription-content'}) != None:
            for j in soup.find('div',{'class':'ProductDescription-content'}).find_all('li'):
                material_item.append(j.text.strip())
            material.append(material_item)
        else:
            material.append(None)

        if soup.find('li',{'class':'ProductDescriptionExtras-item'}) != None:
            color.append(soup.find('li',{'class':'ProductDescriptionExtras-item'}).text.strip().replace("Colour:",''))
    else:
        color.append(None)
        product_description.append(None)
        material.append(None)

    picture_name.append('topshop-men_' + str(counter) + '.jpg') # change variable name
    counter += 1
driver.close()

In [7]:
# data cleaning for material
material_clean = []
for i,v in enumerate(material):
    if v != None:
        item_material = []
        for j in v:
            if "%" in j:
                item_material.append(j)
        material_clean.append(item_material)
    else:
        material_clean.append(None)


In [8]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", 'picture_name']
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = product_name
second_pass.product_description = product_description
second_pass.price = original_price
second_pass.sales_price = sale_price
second_pass.color = color
second_pass.material = material_clean
second_pass.product_url = product_url
second_pass.picture_url = image_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,topshop,Men's Clothing,Camel Classic T-Shirt,"[Classic fit, Crew neck, Short sleeves, 100% C...",None,$10.00,Brown,[100% Cotton],https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM71B36ACAM...,2020-12-24,topshop-men_1.jpg
1,topshop,Men's Clothing,Black Spray On Skinny Jeans,[Our black spray on skinny jeans will always b...,None,$60.00,Black,"[70% Cotton, 28% Polyester, 2% Elastane.]",https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM69F55NBLK...,2020-12-24,topshop-men_2.jpg
2,topshop,Men's Clothing,CONSIDERED Burgundy Twist Essential Knitted Sw...,"[Classic fit, Crew neck, Long sleeves, Essenti...",None,$35.00,Red,[100% Cotton],https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM81B14VBRG...,2020-12-24,topshop-men_3.jpg
3,topshop,Men's Clothing,Black Classic T-Shirt,"[Classic fit, Crew neck, Short sleeves, 100% C...",None,$10.00,Black,[100% Cotton],https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM71B01TBLK...,2020-12-24,topshop-men_4.jpg
4,topshop,Men's Clothing,Black Stretch Skinny Jeans,[The stretch fabric allows comfort and sense o...,None,$60.00,Black,"[98% Cotton, 2% Elastane. Machine washable.]",https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM69F54QBLK...,2020-12-24,topshop-men_5.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
595,topshop,Men's Clothing,Gray Skater Jeans,"[Skater fit long rise, loose leg, rigid fabri...",None,$65.00,Grey,[100% Cotton],https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM69H98TGRY...,2020-12-24,topshop-men_596.jpg
596,topshop,Men's Clothing,Yellow Waffle Docker Beanie,"[Classic docker style, Waffle fabrication, Con...",None,$20.00,Yellow,"[70% Acrylic, 30% Polyester Recycled]",https://us.topman.com/en/tmus/product/shoes-an...,https://images.topman.com/i/TopMan/TM56D11CYLW...,2020-12-24,topshop-men_597.jpg
597,topshop,Men's Clothing,Black and White Blur Floral Print Slim Shirt,"[Slim fit, Classic collar shape, Button fasten...",None,$55.00,Multi,[100% Viscose],https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM83U10UBLK...,2020-12-24,topshop-men_598.jpg
598,topshop,Men's Clothing,Black MA2 Bomber Jacket,"[Classic fit, Bomber jacket style, Collar deta...",None,$95.00,Black,[100% Polyester],https://us.topman.com/en/tmus/product/clothing...,https://images.topman.com/i/TopMan/TM64J15UBLK...,2020-12-24,topshop-men_599.jpg


In [9]:
second_pass.to_csv("topshop-men.csv", index=False, encoding = 'utf-8-sig')

In [26]:
import os
import requests as req

new_pd = pd.read_csv('topshop-men.csv', encoding='utf-8-sig')

os.mkdir('topshop-men')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}

    img_data = req.get(link,headers=headers).content
    with open('topshop-men/'+ "topshop-men_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1